In [ ]:
!pip install mediapipe==0.8.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 13.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import re
import mediapipe as mp
import pandas as pd

In [ ]:
print(mp.__version__)

AttributeError: ignored

In [ ]:
# Initialize mediapipe lib
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

# no_of_frames = 120

In [ ]:
def make_landmark_timestep(results):
    c_lm = []
    for id, lm in enumerate(results.pose_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
        c_lm.append(lm.z)
        c_lm.append(lm.visibility)
    return c_lm

In [ ]:
def draw_landmark_on_image(mpDraw, results, frame):
    # Draw the line to connect the landmarks
    mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

    # Draw the landmarks on the frame
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = frame.shape
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(frame, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
    return frame

In [ ]:
# Set the path of all the videos
video_folder_features = '/content/drive/MyDrive/PBL/PBL5/_Bottom-up-Framework/New_Data/Features'
video_folder_data = '/content/drive/MyDrive/PBL/PBL5/data_resize_30'
# video_folder_resize = '/content/drive/MyDrive/PBL/PBL5/yuudemo/Resize'


### Read video from the folder and save the landmarks to a csv file

#### Preprocessing video

Resize video

In [ ]:
def resize_video(video_path, output_path, new_size=(720, 480), new_fps=20):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print("fps: ", fps)
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    # print("frame_size: ", frame_size)
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), new_fps, new_size)

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            # Resize frame
            resized_frame = cv2.resize(frame, new_size)
            writer.write(resized_frame)
        else:
            break

    cap.release()
    writer.release()



Create folder resize to save video resized (Only run one time to have data, no need to run more)

In [ ]:
# for root, dirs, files in os.walk(video_folder_data):
#     for dir in dirs:
#         # if (dir.startswith('Fall') or dir.startswith('Walk')):
#         #     print(dir)
#         for video in os.listdir(os.path.join(root, dir)):
#             video_path = os.path.join(root, dir, video)
#             # print(video_path)
#             output_folder = os.path.join('/home/yuu/Documents/PBL5-demo/Resize', dir)
#             if not os.path.exists(output_folder):
#                 os.makedirs(output_folder)
#             output_path = os.path.join(output_folder, video.replace('.mp4', '_resized.mp4'))
#             # print(output_path)
#             resize_video(video_path, output_path)


Cut video resized into frames -> save image

In [ ]:
# def extract_frames_from_video(video_path, output_folder, frames_per_second):
#     cap = cv2.VideoCapture(video_path)
#     frame_rate = cap.get(cv2.CAP_PROP_FPS)
#     total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     count = 0
#     index = 0

#     while cap.isOpened():
#         ret, frame = cap.read()

#         if ret:
#             count += 1
#             if count < frame_rate * 3:
#                 continue
#             elif count % int(frame_rate / frames_per_second) == 0:
#                 index += 1
#                 output_path = os.path.join(output_folder, f"{index:04d}.jpg")
#                 cv2.imwrite(output_path, frame)
#             elif index > 0 and count >= (index * frame_rate / frames_per_second + frame_rate * 6):
#                 break
#         else:
#             break

#     cap.release()


Cut video into image (No need to run after having data)

In [ ]:
# for root, dirs, files in os.walk(video_folder_resize):
#     for dir in dirs:
#         video_folder = os.path.join(root, dir)
#         output_folder = os.path.join('/home/yuu/Documents/PBL5-demo/Image', dir)
#         for video in os.listdir(video_folder):
#             video_path = os.path.join(video_folder, video)
#             output_path = os.path.join(output_folder, video.replace('_resized.mp4', ''))
#             # print(output_path)
#             extract_frames_from_video(video_path, output_path, 6)

### Read video and save landmark

Save the landmarks to a csv file

In [ ]:
def save_landmark_to_csv(label, lm_list):
    df = pd.DataFrame(lm_list)
    df.to_csv(label + '.csv')
    print('File name:', label + '.csv')

In [ ]:
def handle_video_features(video_path, output_path, lm_list):
    cap = cv2.VideoCapture(video_path)

    if (cap.isOpened() == False):
        print("Error opening video stream or file")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    # print("output_path: ", output_path)
    # writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            # Recognize the pose
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frameRGB)

            if results.pose_landmarks:
                # Read the value of the landmarks
                lm = make_landmark_timestep(results)
                lm_list.append(lm)

                # Draw the landmarks on the frame
                frame = draw_landmark_on_image(mpDraw, results, frame)

            # cv2.imshow('frame', frame)
            # writer.write(frame)
            if cv2.waitKey(1) == ord('q'):
                break


        else:
            break

    cap.release()
    # writer.release()
    cv2.destroyAllWindows()


Code to create folder Features (No need to run after having data)

In [ ]:
for root, dirs, files in os.walk(video_folder_data):
    for dir in dirs:
        lm_list = []
        for video in os.listdir(os.path.join(root, dir)):
            video_path = os.path.join(root, dir, video)
            # print(video_path)
            output_folder = os.path.join(video_folder_features, dir)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            output_path = os.path.join(output_folder, video.replace('.mp4', '_features.mp4'))
            # print(output_path)
            handle_video_features(video_path, output_path, lm_list)
        save_landmark_to_csv(dir, lm_list)

File name: walk.csv
